In [5]:
project_id = input("Please enter the project you'd like to work in\n")

Please enter the project you'd like to work in
 bsd-specialolymp


In [6]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

In [19]:
proceed = True
while proceed:
    origin_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone from:\n")
    target_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone to:\n")
    dataset_id = f'{project_id}.{origin_schema}'

    tables = client.list_tables(dataset_id)  # Make an API request.

    print("Tables contained in '{}':".format(dataset_id))
    for table in tables:
        print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        if table.table_type == 'VIEW':
            clone_table_statement=f"""
            CREATE OR REPLACE VIEW
              {project_id}.{target_schema}.{table.table_id}
              AS SELECT * FROM {project_id}.{origin_schema}.{table.table_id}
            """
        else:
            clone_table_statement=f"""
            CREATE OR REPLACE TABLE
              {project_id}.{target_schema}.{table.table_id}
              CLONE {project_id}.{origin_schema}.{table.table_id}
            """
        print(client.query(clone_table_statement).result())
        
    proceed = input("\nIf you'd like to clone another schema please enter 'yes'; only the full word yes will be accepted")=='yes'

Please enter the schema (called 'dataset' in BigQuery) you'd like to clone from:
 prod_bsd_rep
Please enter the schema (called 'dataset' in BigQuery) you'd like to clone to:
 clone_test


Tables contained in 'bsd-specialolymp.prod_bsd_rep':
bsd-specialolymp.prod_bsd_rep.cloudaudit_googleapis_com_activity
bsd-specialolymp.prod_bsd_rep.cloudaudit_googleapis_com_data_access
bsd-specialolymp.prod_bsd_rep.meta_billing
bsd-specialolymp.prod_bsd_rep.ref_mailing_details
bsd-specialolymp.prod_bsd_rep.rep_channel_report
bsd-specialolymp.prod_bsd_rep.rep_client_mailing_segment_summary
bsd-specialolymp.prod_bsd_rep.rep_client_mailing_summary
bsd-specialolymp.prod_bsd_rep.rep_mailing_summary
bsd-specialolymp.prod_bsd_rep.rep_nonrecurring_mailing_summary
bsd-specialolymp.prod_bsd_rep.rep_recurring_mailing_summary



If you'd like to clone another schema please enter 'yes'; only the full word yes will be accepted 
